In [1]:
import sys
import warnings
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.metrics import r2_score

**NOTE!!!!**

- The number of predictors (input features) for the $\, R^2 \,$ calculation is fixed at $\, \boldsymbol{53}. \,$
- This can be changed by altering the $\, \text{p} \,$ parameter of the $\, \text{adjusted_R2} \,$ function.

<h3>Helper functions</h3>

In [2]:
def make_cmap(colors, position=None, bit=False):
    '''
    The RGB values may either be in 8-bit [0 to 255] (in which bit must be set to True when called) 
    or arithmetic [0 to 1] (default). 
    
    Args:
    colors -- a list containing RGB values
    
    Returns:
    
    Notes:
    - The RGB values may either be in 8-bit
    - If 8-bit RGB values are used, the 'bit' argument must be set to True.
    
    make_cmap returns
    a cmap with equally spaced colors.
    Arrange your tuples so that the first color is the lowest value for the
    colorbar and the last is the highest.
    position contains values from 0 to 1 to dictate the location of each color.
    '''
    bit_rgb = np.linspace(0, 1, 256)
    if position is None:
        position = np.linspace(0, 1, len(colors))
    else:
        if len(position) != len(colors):
            sys.exit("position length must be the same as colors")
        elif position[0] != 0 or position[-1] != 1:
            sys.exit("position must start with 0 and end with 1")
    if bit:
        for i in range(len(colors)):
            colors[i] = (bit_rgb[colors[i][0]],
                         bit_rgb[colors[i][1]],
                         bit_rgb[colors[i][2]])
    cdict = {'red': [], 'green': [], 'blue': []}
    for pos, color in zip(position, colors):
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))

    cmap = mpl.colors.LinearSegmentedColormap('my_colormap', cdict, 256)
    return cmap

In [3]:
cmap2Dhist = make_cmap([(255, 255, 255), (127, 188, 227), (82, 170, 115), (225, 189, 74), (222, 64, 39), (149, 21, 25)], bit=True)

In [4]:
def adjusted_R2(y_true, y_pred, p=53):
    """
    Calculates the adjusted R-squared (R2) value.
    
    R-squared: R2 measures the proportion of the variance in the dependent variable that is predictable 
               from the independent variables. R2 indicates how well the model fits the data.
               R2 is useful when using a single predictor. However, when using multiple predictors,
               R2 will always increase as more predictors are added, even if those predictors do not
               actually  contribute to the model's predictive power.
               
    Adjusted R-squared: Adjusted R2 modifies the R2 value by accounting for the number of predictors 
                        in the model relative to the number of data points. 
                        Adjusted R2 adjusts for the degrees of freedom, penalizing the addition of 
                        non-significant predictors. Adjusted R2 is more appropriate when performing
                        regression with multiple predictors.
                        
    Args:
    y_true -- the true y-values
    y_pred -- the y-values predicted by the model
    p -- the number of predictors in the model
    
    Returns:
    R2_adj -- adjusted R2 value.
    """
    n = y_true.size    # The number of observations
    R2 = r2_score(y_true, y_pred)
    R2_adj = 1 - (((1 - R2) * (n - 1)) / (n - p - 1))
    return R2_adj

In [5]:
def computeStats(true, predicted, computeEEratio=False):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', message='invalid value encountered in true_divide')  # temporarily ignore this warning (would give warnings if predicted only contains a single value)
        N = len(true)
        R = np.corrcoef(true, predicted)[0, 1]
        #R2 = R**2  # r2_score(true, predicted)
        R2_adj = adjusted_R2(true, predicted)
        RMSE = np.sqrt(np.mean((true - predicted)**2))
        BIAS = np.median(predicted - true)
        #EE = None
        if computeEEratio:
            EE = np.logical_and(predicted >= true * 0.85 - 0.05, predicted <= true * 1.15 + 0.05).sum() / N * 100.0
    return R, R2_adj, RMSE, BIAS

In [6]:
def calculate_errorbars_in_range(x,y,start_val,stop_val,step):
    xx = np.arange(start_val,stop_val,step)
    
    x_loc = np.zeros(xx.size-1)
    y_mean = np.zeros(xx.size-1)
    y_std = np.zeros(xx.size-1)
    
    for i in range(np.size(xx)-1):
        ind = np.where(np.logical_and(x >= xx[i], x < xx[i+1]))
        
        vals = y[ind]
        
        if vals.size == 0:
            y_mean[i] = np.nan
            y_std[i] = np.nan
        else:
            y_mean[i] = np.nanmean(vals)
            y_std[i] = np.nanstd(vals)
            
        x_loc[i] = np.nanmean((xx[i],xx[i+1]))

    return x_loc, y_mean, y_std

<h3>Main plotting function</h3>

In [7]:
def plot_data(y_true, y_pred, title, x_label, y_label, error_bar_on=True, save_fig=False, fig_name=None):
    """
    Args:
    y_true -- true output values
    y_pred -- predicted output values
    title -- title name for plot
    x_label --  x-axis label for plot
    y_label -- y-xis label for plot
    error_bar_on -- if True, plots the error bar
    save_fig -- if True, saves the figure in the current working directory
    """
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)
    
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)
        
    y_true = y_true.ravel()
    y_pred = y_pred.ravel()
    
    y_true_min = np.min(y_true) - 0.5
    y_true_max = np.max(y_true) + 0.5
    y_pred_min = np.min(y_pred) - 0.5
    y_pred_max = np.max(y_pred) + 0.5
    
    N = int(np.size(y_true))
    
    fig = plt.figure(figsize=(8, 6), dpi=100)
    ax = fig.subplots()
    
    #ax.scatter(y_true, y_pred, zorder=1)
        
    norm2Dhist = LogNorm(vmin=1, vmax=int(0.025 * N))
    
    ax.hist2d(y_true, y_pred, bins=(64,64), range=[[y_true_min, y_true_max],[y_pred_min, y_pred_max]], cmap=cmap2Dhist, norm=norm2Dhist, alpha=0.8, zorder=2)
    ax.grid(True)
    
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.set_title(title)
    ax.set_ylim([y_pred_min, y_pred_max])
    ax.set_xlim([y_true_min, y_true_max])
    
    ax.plot([y_true_min, y_true_max], [y_true_min, y_true_max], "k-", linewidth=2.0, alpha=0.8, zorder=3)
        
    axCB = fig.add_axes([0.935, 0.1, 0.02, 0.85])
    cb1 = mpl.colorbar.ColorbarBase(axCB, cmap=cmap2Dhist, norm=norm2Dhist, orientation='vertical')
    cb1.set_label('N', fontsize=24)
    
    if error_bar_on:
        x_error_loc, y_mean, y_std = calculate_errorbars_in_range(y_true, y_pred, np.min(y_true), np.max(y_true), 1)
        ax.errorbar(x_error_loc, y_mean, y_std,marker='o',ecolor="m",markerfacecolor='m',linestyle='None',capsize=5,zorder=4)
    
    R, R2, RMSE, BIAS = computeStats(y_true, y_pred)
    
    ax.text(0.99, 0.08 + 4 * 0.08, 'N={}'.format(N), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    #ax.text(0.99, 0.08 + 4 * 0.08, 'R={:.3f}'.format(R), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    ax.text(0.99, 0.08 + 3 * 0.08, 'R2 (adjusted)={:.3f}'.format(R2), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    ax.text(0.99, 0.08 + 2 * 0.08, 'RMSE={:.3f}'.format(RMSE), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    ax.text(0.99, 0.08 + 1 * 0.08, 'BIAS={:.3f}'.format(BIAS), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    #ax.text(0.99, 0.08 + 0 * 0.08, 'EE={}'.format(EE), horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, fontsize=18, color='k', zorder=99)#, bbox={'facecolor': 'white', 'alpha': 0.7, 'boxstyle': 'square,pad=0.03'})
    
    if save_fig:
        if fig_name is None:
            raise ValueError("fig_name must be provided if save_fig is True")
        fig.savefig(fig_name + ".png")

<h3>Convert into .py file</h3>

In [8]:
!jupyter nbconvert --to script visualization_utility.ipynb

[NbConvertApp] Converting notebook visualization_utility.ipynb to script
[NbConvertApp] Writing 9172 bytes to visualization_utility.py


- Jupyter notebooks need to be converted to python scripts ('.py' files) to be imported directly into another notebook / script.
- It seems that this did not convert this notebook into a .py file, but rather made a copy of this notebook as a .py script.